In [ ]:
!mkdir ./U2-net/inputdata

# Data 설명 

./U2-net/inputdata/train폴더안에 인풋 이미지들을 넣어 주시면됩니다.

./ 경로에 json 파일 넣어주시면 됩니다.

# 1. json -> csv 컨버터

In [165]:
import json
import numpy as np
import cv2
from PIL import Image
import pandas as pd

with open("labels_my-project-name.json", "r") as st_json:
    data = json.load(st_json)


In [166]:
imageid = []
width = []
height = []
classid = []
rle = []
sdict = {}

for i in range(len(data['annotations'])):
    imageid.append(data['images'][i]['file_name'])
    width.append(data['images'][i]['width'])
    height.append( data['images'][i]['height'])
    classid.append(data["annotations"][i]['category_id'])


    segmentation_points = data["annotations"][i]["segmentation"][0]
    bbox = data["annotations"][i]["bbox"]
    segmentation_array = np.array(segmentation_points).reshape((-1, 2)).astype(np.int32)
    image_width = data["images"][i]["width"]
    image_height = data["images"][i]["height"]
    canvas = np.zeros((image_height, image_width), dtype=np.uint8)
    cv2.fillPoly(canvas, [segmentation_array], color=(255, 255, 255))
    output_file_name = "segmentation_image.jpg"
    cv2.imwrite(output_file_name, canvas)
    
    def binarize_image(image):
        
        grayscale_image = image.convert('L')
        
        threshold = 128  
        binary_image = grayscale_image.point(lambda x: 0 if x < threshold else 255, '1')
        return binary_image
    def run_length_encode(mask):
        rle = []
        count = 0
        is_running = False
       
        for pixel in mask.getdata():
            if pixel == 255:  
                if not is_running:
                    start = count + 1 
                    is_running = True
            else:
                if is_running:
                    length = count - start + 1 
                    rle.append((start, length))
                    is_running = False
            count += 1
        if is_running:
            length = count - start + 1
            rle.append((start, length))
        return rle
    def run_length_decode(rle, mask_shape):
        mask = Image.new('L', mask_shape)
        for start, length in rle:
            for i in range(length):
                x = (start + i) % mask_shape[0]
                y = (start + i) // mask_shape[0]
                mask.putpixel((x, y), 255)  
        return mask
    image = Image.open('segmentation_image.jpg')
    binary_image = binarize_image(image)
    rle_encoded = run_length_encode(binary_image)
    rle_encoded = ' '.join([' '.join([str(i[0]),str(i[1])]) for i in rle_encoded])
    rle.append(rle_encoded)

In [170]:
import pandas as pd

f = pd.DataFrame({'ImageId':imageid,
                    'EncodedPixels': rle,
                    'Width':width, 
                    'Height':height, 
                    'ClassId':classid})

f.to_csv("./U2-net/inputdata/train.csv",index=False)

# 아래부터는 정리 코드

In [6]:
# torch 설치
# !conda install pytorch==1.13.1 torchvision==0.14.1 torchaudio==0.13.1 pytorch-cuda=11.7 -c pytorch -c nvidia -y

In [4]:
# git 설치
!conda install git -y

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 22.9.0
  latest version: 23.3.1

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/ohseunghwan/anaconda3/envs/soomgo

  added / updated specs:
    - git


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    curl-7.88.1                |       h5eee18b_0          88 KB
    gdbm-1.18                  |       hd4cb3f1_4         194 KB
    gettext-0.21.0             |       h39681ba_1         3.0 MB
    git-2.34.1                 | pl5262hc120c5b_0         6.0 MB
    libxml2-2.10.3             |       hcbfbd50_0         755 KB
    pcre2-10.37                |       he7ceb23_1         839 KB
    perl-5.34.0                |       h5eee18b_2        12.4 MB
    ------------------------------------------------------------
    

In [1]:
!git clone https://github.com/shiny0510/U2-net.git

Cloning into 'U2-net'...
remote: Enumerating objects: 57, done.
remote: Counting objects: 100% (57/57), done.
remote: Compressing objects: 100% (53/53), done.
remote: Total 57 (delta 3), reused 56 (delta 2), pack-reused 0
Receiving objects: 100% (57/57), 2.40 MiB | 5.37 MiB/s, done.
Resolving deltas: 100% (3/3), done.


In [29]:
!pip install pyyaml
!pip install opencv-python
!conda install tensorboard
!pip install pandas

  Using cached PyYAML-6.0-cp38-cp38-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (701 kB)


### U2-net/inputdata/train 경로안에 이미지 데이터와 

### U2-net/inputdata/train.csv를 만드셨다면, 그냥 돌리시면 됩니다

In [176]:
!python ./U2-net/train.py 

name training_cloth_segm_u2net_exp1
image_folder ./cloth-segmentation/inputdata/train/
df_path ./cloth-segmentation/inputdata/train.csv
distributed False
isTrain True
fine_width 768
fine_height 768
mean 0.5
std 0.5
batchSize 2
nThreads 2
max_dataset_size inf
serial_batches False
continue_train False
save_freq 1000
print_freq 10
image_log_freq 100
iter 100000
lr 0.0002
clip_grad 5
logs_dir logs/training_cloth_segm_u2net_exp1
save_dir results/training_cloth_segm_u2net_exp1
cuda:0
dataset [AlignedDataset] was created
100%|████████████████████████████████████████| 50/50 [00:00<00:00, 12529.29it/s]
Total number of images avaliable for training: 50
Entering training loop!
/home/ohseunghwan/anaconda3/envs/soomgo/lib/python3.8/site-packages/torch/nn/functional.py:3734: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")
'[step-00000000] [time-6.141] [total_loss-